<html>
<center>
</h4>
<font color='blod' size="+2">Task Description!</font></h4>
</center>
<p>In this Notebook, we will fine-tune the models BERT and BioBERT for the NER test on JNLPBA corpora.
<!-- TODO(b/319266067) remove temporary advert after a few weeks. -->
<div class="markdown-google-sans">
  <h2>Biomedical corpora : </h2>
  <ul>
  <li><a href="https://huggingface.co/datasets/ncbi_disease#dataset-card-for-ncbi-disease">NCBI Disease</a></li>
  <p>This dataset contains the disease name and concept annotations of the NCBI disease corpus, a collection of 793 PubMed abstracts fully annotated at the mention and concept level to serve as a research resource for the biomedical natural language processing community.</p>
  <div>
  <center>
  <table border="1">
  <tr>
    <th>Dataset</th>
    <th>Train</th>
    <th>Validation</th>
    <th>Test</th>
  </tr>
  <tr>
    <td>Instances</td>
    <td>5433</td>
    <td>924</td>
    <td>941</td>
  </tr>
</table>
</div>
  <li><a href="https://huggingface.co/datasets/jnlpba">JNLPBA</a></li>
   The data came from the GENIA version 3.02 corpus (Kim et al., 2003). This was formed from a controlled search on MEDLINE using the MeSH terms human, blood cells and transcription factors. From this search 2,000 abstracts were selected and hand annotated according to a small taxonomy of 48 classes based on a chemical classification. Among the classes, 36 terminal classes were used to annotate the GENIA corpus.
   <div>
   <center>
   <table border="1">
  <tr>
    <th>Dataset</th>
    <th>Train</th>
    <th>Validation</th>
  </tr>
  <tr>
    <td>Instances</td>
    <td>37094</td>
    <td>7714</td>
  </tr>
</table>

   </div>
   <li><a href="https://huggingface.co/datasets/linnaeus">LINNAEUS</a></li>
   The LINNAEUS corpus consists of 100 full-text documents from the PMCOA document set which were randomly selected. All mentions of species terms were manually annotated and normalized to the NCBI taxonomy IDs of the intended species.
   <div>
   <center>
   <table border="1">
  <tr>
    <th>Dataset</th>
    <th>Train</th>
    <th>Validation</th>
    <th>Test</th>
  </tr>
  <tr>
    <td>Instances</td>
    <td>11936</td>
    <td>4079</td>
    <td>7143</td>
  </tr>
</table>

   </div>
  <li><a href="https://huggingface.co/datasets/bc2gm_corpus/viewer/bc2gm_corpus">BC2GM</a></li>
  Created by Smith et al. at 2008, the BioCreative II Gene Mention Recognition (BC2GM) Dataset contains data where participants are asked to identify a gene mention in a sentence by giving its start and end characters. The training set consists of a set of sentences, and for each sentence a set of gene mentions (GENE annotations)
  <div>
  <center>
  <table border="1">
  <tr>
    <th>Dataset</th>
    <th>Train</th>
    <th>Validation</th>
    <th>Test</th>
  </tr>
  <tr>
    <td>Instances</td>
    <td>12501</td>
    <td>2501</td>
    <td>5001</td>
  </tr>
</table>

  </div>
  <li><a href="https://huggingface.co/datasets/chintagunta85/bc4chemd">BC4CHEMD</a></li>
      BC4CHEMD is a collection of 10,000 PubMed abstracts that contain a total of 84,355 chemical entity mentions labeled manually by expert chemistry literature curators
 <div>
   <center>
   <table border="1">
  <tr>
    <th>Dataset</th>
    <th>Train</th>
    <th>Validation</th>
      <th>Test</th>
  </tr>
  <tr>
    <td>Instances</td>
    <td>30683</td>
    <td>30640</td>
      <td>26365</td>
  </tr>
</table>

   </div>
  </ul>

</div>
</html>

# BERT

<font color='green'>Preparing the data</font>
<p>First things, we need a dataset suitable for token classification. In this section we will use the NCBI Disease data.</p>
<p>To load the NCBI Disease  dataset, we use the load_dataset() method from the huggingface 🤗 Datasets library</p>

In [1]:
!pip install datasets
!pip install evaluate
!pip install transformers
!pip install seqeval
import evaluate
import numpy as np
import torch
from accelerate import Accelerator
from transformers import TrainingArguments
from transformers import Trainer
from transformers import AutoModelForTokenClassification
from transformers import DataCollatorForTokenClassification
from transformers import AutoTokenizer
from datasets import load_dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 895.5 kB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 647.4 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=98d59e7d421f1887f82bbce7f2cf319c25dfd039e46a0acbc9761b93d10f0fdd
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


2024-04-15 18:47:21.113067: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-15 18:47:21.113207: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-15 18:47:21.268973: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:


raw_datasets = load_dataset("bc2gm_corpus")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/12501 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2501 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5001 [00:00<?, ? examples/s]

Dataset bc2gm_corpus downloaded and prepared to /root/.cache/huggingface/datasets/bc2gm_corpus/bc2gm_corpus/1.0.0/198658a8f6102c50f2230023a1fc49526139e3955e86bb62fafb48edfbbaca60. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 12501
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2501
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 5001
    })
})

**Data Fields**

    id: Sentence identifier.
    tokens: Array of tokens composing a sentence.
    ner_tags: Array of tags, where 0 indicates no disease mentioned, 1 signals the first token of a disease and 2 the subsequent disease tokens.


Let’s have a look at the first element of the training set:

In [4]:
print(raw_datasets["train"][0]["tokens"])

['Comparison', 'with', 'alkaline', 'phosphatases', 'and', '5', '-', 'nucleotidase']


Let’s have a look at the first Annotation of the training set:

In [5]:
raw_datasets["train"][0]["ner_tags"]

[0, 0, 1, 2, 0, 1, 2, 2]

In [6]:

ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-GENE', 'I-GENE'], id=None), length=-1, id=None)

In [7]:

label_names = ner_feature.feature.names
label_names

['O', 'B-GENE', 'I-GENE']

<font color='green'>Processing the data</font>

To begin, let’s create a tokenizer object.

we will be using a BERT pretrained model, so we’ll start by downloading and caching the associated tokenizer:


In [8]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

we will tokenize the first token in our corpus

In [9]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
print(inputs.tokens())

['[CLS]', 'Co', '##mp', '##aris', '##on', 'with', 'al', '##kal', '##ine', 'p', '##hos', '##pha', '##tase', '##s', 'and', '5', '-', 'n', '##uc', '##leo', '##ti', '##das', '##e', '[SEP]']


As we can see, the tokenizer added the special tokens used by the model ([CLS] at the beginning and [SEP] at the end)

The  Identification, was tokenized into 3 subwords, I,  ##dent and ##ification'. This introduces a mismatch between our inputs and the labels:

In [10]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True,is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [11]:

tokenized_datasets = raw_datasets.map(tokenize_and_align_labels,batched=True)

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [12]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 12501
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2501
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 5001
    })
})

In [13]:
print(tokenized_datasets['train'][0]['input_ids'])
print(len(tokenized_datasets['train'][0]['input_ids']))

[101, 3291, 8223, 26464, 1320, 1114, 2393, 14046, 2042, 185, 15342, 20695, 25347, 1116, 1105, 126, 118, 183, 21977, 26918, 3121, 9028, 1162, 102]
24


In [14]:

print(tokenized_datasets['train'][0]['labels'])
print(len(tokenized_datasets['train'][0]['labels']))

[-100, 0, -100, -100, -100, 0, 1, -100, -100, 2, -100, -100, -100, -100, 0, 1, 2, 2, -100, -100, -100, -100, -100, -100]
24


In [15]:
example = tokenized_datasets['train'][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
len(tokens),len(tokenized_datasets['train'][0]['labels'])

(24, 24)

In [16]:

tokenized_datasets=tokenized_datasets.remove_columns(['id', 'tokens', 'ner_tags'])
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 12501
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2501
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 5001
    })
})

DataCollatorWithPadding  only pads the inputs (input IDs, attention mask, and token type IDs). Here our labels should be padded the exact same way as the inputs so that they stay the same size, using -100 as a value so that the corresponding predictions are ignored in the loss computation.

In [17]:


data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [18]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    0, -100, -100, -100,    0,    1, -100, -100,    2, -100, -100,
         -100, -100,    0,    1,    2,    2, -100, -100, -100, -100, -100, -100],
        [-100,    0, -100, -100, -100, -100,    0,    0,    0, -100, -100,    0,
         -100, -100, -100, -100, -100, -100,    0, -100, -100, -100, -100, -100]])

Metrics

To have the Trainer compute a metric every epoch, we will need to

 define a compute_metrics() function that takes the arrays of

 predictions and labels, and returns a dictionary with the metric names and values.

The traditional framework used to evaluate token classification prediction is seqeval. To use this metric, we first need to install the seqeval library:

In [19]:

metric = evaluate.load("seqeval")

In [20]:


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [120]:

labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['O', 'O', 'B-GENE', 'I-GENE', 'O', 'B-GENE', 'I-GENE', 'I-GENE']

In [21]:

id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [22]:
id2label

{0: 'O', 1: 'B-GENE', 2: 'I-GENE'}

In [23]:
label2id

{'O': 0, 'B-GENE': 1, 'I-GENE': 2}

**Defining the model**

In [24]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
#just check the number of labels
model.config.num_labels

3

In [26]:

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()


#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


**Fine-tuning the model**

In [27]:

args = TrainingArguments(
    "disease",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    optim='adamw_torch',
    num_train_epochs=8,
    weight_decay=0.01,
)

In [28]:

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    
)
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.083145,0.679060,0.776759,0.724631,0.967790
2,0.119500,0.074408,0.728083,0.800949,0.762779,0.971589
3,0.054700,0.079787,0.740577,0.795257,0.766944,0.971833
4,0.033500,0.090796,0.742675,0.809486,0.774643,0.972655
5,0.033500,0.096837,0.747594,0.810593,0.777820,0.972593
6,0.020800,0.104629,0.760815,0.809170,0.784248,0.973450
7,0.014400,0.111288,0.767456,0.799368,0.783087,0.974147
8,0.010300,0.115032,0.762526,0.803636,0.782542,0.974056


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=3128, training_loss=0.040886521453747666, metrics={'train_runtime': 1466.5967, 'train_samples_per_second': 68.191, 'train_steps_per_second': 2.133, 'total_flos': 5242377370754250.0, 'train_loss': 0.040886521453747666, 'epoch': 8.0})

In [29]:
eval_metrics = trainer.state.log_history

In [30]:
eval_metrics[-2]

{'eval_loss': 0.11503172665834427,
 'eval_precision': 0.7625262526252625,
 'eval_recall': 0.8036363636363636,
 'eval_f1': 0.7825417596797783,
 'eval_accuracy': 0.9740563900602934,
 'eval_runtime': 26.606,
 'eval_samples_per_second': 187.965,
 'eval_steps_per_second': 5.901,
 'epoch': 8.0,
 'step': 3128}

# Biobert

In [31]:
biobert_model_checkpoint = "dmis-lab/biobert-v1.1"
biobert_tokenizer = AutoTokenizer.from_pretrained(biobert_model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [32]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = biobert_tokenizer(examples["tokens"], truncation=True,is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [33]:

tokenized_datasets_biobert = raw_datasets.map(tokenize_and_align_labels,batched=True)

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [34]:
tokenized_datasets_biobert=tokenized_datasets_biobert.remove_columns(['id', 'tokens', 'ner_tags'])
tokenized_datasets_biobert


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 12501
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2501
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 5001
    })
})

In [35]:
data_collator = DataCollatorForTokenClassification(tokenizer=biobert_tokenizer)


model = AutoModelForTokenClassification.from_pretrained(
    biobert_model_checkpoint,num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
)

args = TrainingArguments(
    "disease1",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    optim='adamw_torch',
    num_train_epochs=8,
    weight_decay=0.01,
)


pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets_biobert["train"],
    eval_dataset=tokenized_datasets_biobert["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=biobert_tokenizer,
    
)
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.058676,0.775180,0.817708,0.795876,0.977507
2,0.092700,0.058943,0.793887,0.833676,0.813295,0.978322
3,0.037300,0.061237,0.802858,0.835099,0.818661,0.978524
4,0.022900,0.069247,0.820386,0.839842,0.830000,0.979535
5,0.022900,0.078889,0.810670,0.836047,0.823163,0.979305
6,0.013200,0.090746,0.812729,0.839842,0.826063,0.979159
7,0.008900,0.098175,0.806280,0.840316,0.822947,0.978887
8,0.005700,0.101433,0.813315,0.838261,0.825600,0.979054


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=3128, training_loss=0.029112372918964347, metrics={'train_runtime': 1430.8616, 'train_samples_per_second': 69.894, 'train_steps_per_second': 2.186, 'total_flos': 5242377370754250.0, 'train_loss': 0.029112372918964347, 'epoch': 8.0})